<a href="https://colab.research.google.com/github/GEANCUNHA/datasetMVP/blob/main/MVPGeanCunha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Utilizei para este MVP um banco de dados do Kaggle. Esse banco de dados apresenta o historico de solicitações de créditos (empréstimo) em um banco com seus respectivos status (aprovado e reprovado). O meu modelo utilizará esses dados para treinamento e assim decidir sobre sobre novas solicitações.

**Importação e pré-processamento dos dados**

In [70]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np

# Carregar o dataset diretamente do repositorio do Github
url = 'https://raw.githubusercontent.com/GEANCUNHA/datasetMVP/refs/heads/main/loan_approval_dataset.csv'
dados = pd.read_csv(url)

# Remover o campo loan_id antes de treinar o modelo - este campo está no dataset mas vi que ele poderia atrapalhar o modelo, visto ser apenas um dado de controle de solicitações.
dados = dados.drop(columns=['loan_id'])

# Remover espaços em branco nos nomes das colunas - os dados nao estavam bem tratados, alguns titulos de coluna estavam com espaços, isso atrapalhou muito no codigo, em alguns momentos tive que fazer alguns tratamentos
dados.columns = dados.columns.str.strip()


# Exibir as primeiras linhas do dataset para ver as colunas e entender os dados
dados.head(5)




,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


**Continuação do pré-processamento**

In [71]:

# Agora, converter colunas categóricas usando get_dummies (One Hot Encoding) - essas duas colunas eram de strings, portando fiz a conversão para números para facilitar o treinamento.
dados = pd.get_dummies(dados, columns=['education', 'self_employed'], drop_first=True)

# Remover espaços e caracteres especiais dos nomes das colunas
dados.columns = dados.columns.str.replace(' ', '_')

dados['education__Not_Graduate'] = dados['education__Not_Graduate'].astype(int)
dados['self_employed__Yes'] = dados['self_employed__Yes'].astype(int)

# Exibir as primeiras linhas para ver as mudanças
dados.head(5)



,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status,education__Not_Graduate,self_employed__Yes
0,2,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved,0,0
1,0,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected,1,1
2,3,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected,0,0
3,3,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected,0,0
4,5,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected,1,1


**Separação entre treino e teste e normalização/padronização dos dados**

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Separar o dataset em X e y
X = dados.drop('loan_status', axis=1)
y = dados['loan_status']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# ---------------------------
# Aplicar Padronização
# ---------------------------
scaler = StandardScaler()
X_train_padronizado = scaler.fit_transform(X_train)
X_test_padronizado = scaler.transform(X_test)

# ---------------------------
# Aplicar Normalização
# ---------------------------
normalizer = MinMaxScaler()
X_train_normalizado = normalizer.fit_transform(X_train)
X_test_normalizado = normalizer.transform(X_test)

# Verificar as dimensões para checar se houve a divisão entre treino e teste
print(f"Treino padronizado: {X_train_padronizado.shape}, Teste padronizado: {X_test_padronizado.shape}")
print(f"Treino normalizado: {X_train_normalizado.shape}, Teste normalizado: {X_test_normalizado.shape}")




Treino padronizado: (2988, 11), Teste padronizado: (1281, 11)
Treino normalizado: (2988, 11), Teste normalizado: (1281, 11)


**Treinar e avaliar os modelos**

In [77]:
# Importar as bibliotecas necessárias para os modelos e pipelines
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split

# -------------------------------
# Definição de Pipelines
# -------------------------------
# Função para criar pipeline com o modelo e o scaler
def criar_pipeline(modelo, scaler):
    return Pipeline(steps=[('scaler', scaler), ('modelo', modelo)])

# Definir os pipelines para cada modelo com padronização e normalização
pipelines_padronizados = {
    'KNN': criar_pipeline(KNeighborsClassifier(), StandardScaler()),
    'SVM': criar_pipeline(SVC(), StandardScaler()),
    'Naive Bayes': criar_pipeline(GaussianNB(), StandardScaler()),
    'Árvore de Decisão': criar_pipeline(DecisionTreeClassifier(), StandardScaler())
}

pipelines_normalizados = {
    'KNN': criar_pipeline(KNeighborsClassifier(), MinMaxScaler()),
    'SVM': criar_pipeline(SVC(), MinMaxScaler()),
    'Naive Bayes': criar_pipeline(GaussianNB(), MinMaxScaler()),
    'Árvore de Decisão': criar_pipeline(DecisionTreeClassifier(), MinMaxScaler())
}

# -------------------------------
# Cross-Validation
# -------------------------------
# Função para realizar cross-validation e retornar a média e desvio padrão
def realizar_cross_validation_com_desvio(pipeline, X, y):
    scores = cross_val_score(pipeline, X, y, cv=5)  # 5-fold cross-validation
    return scores.mean(), scores.std()

# Avaliar modelos com dados padronizados (incluindo desvio padrão)
print("Resultados com Padronização (Cross-Validation):")
for nome, pipeline in pipelines_padronizados.items():
    media, desvio = realizar_cross_validation_com_desvio(pipeline, X_train, y_train)
    print(f"{nome}: Média = {media:.4f}, Desvio Padrão = {desvio:.4f}")

# Avaliar modelos com dados normalizados (incluindo desvio padrão)
print("\nResultados com Normalização (Cross-Validation):")
for nome, pipeline in pipelines_normalizados.items():
    media, desvio = realizar_cross_validation_com_desvio(pipeline, X_train, y_train)
    print(f"{nome}: Média = {media:.4f}, Desvio Padrão = {desvio:.4f}")

# Predição no Conjunto de Teste para cada modelo
print("\nContagem de Aprovados no Conjunto de Teste para cada Modelo:")

# Avaliar pipelines padronizados
for nome, pipeline in pipelines_padronizados.items():
    # Treinar o pipeline no conjunto de treino
    pipeline.fit(X_train, y_train)

    # Fazer predição no conjunto de teste
    y_pred = pipeline.predict(X_test)

    # Limpar espaços extras nas predições
    y_pred = [pred.strip() for pred in y_pred]

    # Contar quantos foram aprovados ('Approved') e rejeitados ('Rejected')
    aprovados = sum([pred == 'Approved' for pred in y_pred])
    rejeitados = sum([pred == 'Rejected' for pred in y_pred])

    print(f"{nome}: {aprovados} aprovados, {rejeitados} rejeitados")

# Avaliar pipelines normalizados
print("\nContagem de Aprovados no Conjunto de Teste (Modelos Normalizados):")
for nome, pipeline in pipelines_normalizados.items():
    # Treinar o pipeline no conjunto de treino
    pipeline.fit(X_train, y_train)

    # Fazer predição no conjunto de teste
    y_pred = pipeline.predict(X_test)

    # Limpar espaços extras nas predições
    y_pred = [pred.strip() for pred in y_pred]

    # Contar quantos foram aprovados ('Approved') e rejeitados ('Rejected')
    aprovados = sum([pred == 'Approved' for pred in y_pred])
    rejeitados = sum([pred == 'Rejected' for pred in y_pred])

    print(f"{nome}: {aprovados} aprovados, {rejeitados} rejeitados")


Resultados com Padronização (Cross-Validation):
KNN: Média = 0.8842, Desvio Padrão = 0.0107
SVM: Média = 0.9347, Desvio Padrão = 0.0114
Naive Bayes: Média = 0.9227, Desvio Padrão = 0.0101
Árvore de Decisão: Média = 0.9762, Desvio Padrão = 0.0047

Resultados com Normalização (Cross-Validation):
KNN: Média = 0.8879, Desvio Padrão = 0.0128
SVM: Média = 0.9314, Desvio Padrão = 0.0139
Naive Bayes: Média = 0.9227, Desvio Padrão = 0.0101
Árvore de Decisão: Média = 0.9742, Desvio Padrão = 0.0044

Contagem de Aprovados no Conjunto de Teste para cada Modelo:
KNN: 815 aprovados, 466 rejeitados
SVM: 797 aprovados, 484 rejeitados
Naive Bayes: 799 aprovados, 482 rejeitados
Árvore de Decisão: 801 aprovados, 480 rejeitados

Contagem de Aprovados no Conjunto de Teste (Modelos Normalizados):
KNN: 810 aprovados, 471 rejeitados
SVM: 807 aprovados, 474 rejeitados
Naive Bayes: 799 aprovados, 482 rejeitados
Árvore de Decisão: 801 aprovados, 480 rejeitados


**Otimização de hiperparametros para o modelo escolhido (Arvore de decisão com dados normalizados)**

In [78]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Definir os hiperparâmetros que queremos testar
param_grid = {
    'criterion': ['gini', 'entropy'],  # Função de avaliação para divisão
    'max_depth': [3, 5, 10, None],  # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10],  # Mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],  # Mínimo de amostras em um nó folha
    'max_features': [None, 'sqrt', 'log2']  # Número máximo de features consideradas para cada divisão
}

# Criar o modelo base da Árvore de Decisão
arvore = DecisionTreeClassifier()

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=arvore, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Realizar o GridSearch nos dados normalizados
grid_search.fit(X_train_normalizado, y_train)

# Exibir os melhores hiperparâmetros encontrados
print(f"Melhores hiperparâmetros: {grid_search.best_params_}")
print(f"Melhor acurácia: {grid_search.best_score_}")


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Melhores hiperparâmetros: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 10}
Melhor acurácia: 0.9799207856450591


**Treinar o modelo escolhido com os melhores hiperparametros encontrados**

In [97]:

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pickle

# Treinar a Árvore de Decisão com os melhores hiperparâmetros
arvore_otimizada = DecisionTreeClassifier(criterion='entropy', max_depth=10, max_features=None, min_samples_leaf=4, min_samples_split=2)

# Treinar o modelo otimizado com os dados normalizados
arvore_otimizada.fit(X_train_normalizado, y_train)

# Avaliar o modelo nos dados de teste
y_pred = arvore_otimizada.predict(X_test_normalizado)
accuracy_final = accuracy_score(y_test, y_pred)
print(f"Acurácia final nos dados de teste: {accuracy_final:.4f}")

# Limpar espaços extras nas predições (caso as predições sejam strings)
y_pred = [pred.strip() for pred in y_pred]

# Contar quantos foram aprovados ('Approved') e rejeitados ('Rejected')
aprovados = sum([pred == 'Approved' for pred in y_pred])
rejeitados = sum([pred == 'Rejected' for pred in y_pred])

# Exibir os resultados
print(f"Árvore de Decisão Otimizada: {aprovados} aprovados, {rejeitados} rejeitados")

# Exportar o modelo final usando pickle
with open('modelo_arvore_decisao_otimizado.pkl', 'wb') as f:
    pickle.dump(arvore_otimizada, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


Acurácia final nos dados de teste: 0.9852
Árvore de Decisão Otimizada: 812 aprovados, 469 rejeitados


**Análise de resultados do modelo**

O modelo selecionado para este projeto foi a Árvore de Classificação, uma vez que apresentou a melhor acurácia durante o processo de validação cruzada (cross-validation), além de ter exibido o menor desvio padrão entre os modelos testados. Com a otimização de hiperparâmetros, a acurácia foi ainda mais aprimorada, reforçando a escolha do modelo.

Um dos principais desafios enfrentados no treinamento do modelo foi o tratamento dos dados, especificamente a presença de espaços indesejados nas strings, o que causava problemas na correta interpretação das variáveis pela aplicação full stack. Embora o modelo estivesse funcionando adequadamente, a aplicação inicialmente interpretava as predições de maneira incorreta devido a esses problemas com as strings.

Além disso, durante o desenvolvimento, o modelo começou a rotular todas as novas solicitações como "Rejeitado". Após análise, identifiquei que isso ocorria porque os dados fornecidos pelos usuários precisavam ser normalizados para ficarem compatíveis com os dados com os quais o modelo havia sido treinado. Após implementar essa correção, a aplicação passou a realizar predições corretas e consistentes.